<a href="https://colab.research.google.com/github/derekhsu/myparser/blob/master/data_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup
import re
from string import Template

In [0]:
def parse_minanoav(url, name):
  r = requests.get(url)
  if r.status_code != requests.codes.ok:
    print("Unable to connect the specified URL")
    return False
  
  b = BeautifulSoup(r.text)
  birth_text = b.find("span", text="生年月日").find_parent("td").find("p").text
  m = re.search(r"(\d+)年(\d+)月(\d+)日", birth_text)
  birth_year = ""
  birth_month = ""
  birth_day = ""
  if m:
    birth_year = m.group(1)
    birth_month = int(m.group(2))
    birth_day = m.group(3)
  
  size_text = b.find('span', text="サイズ").find_parent("td").find("p").text
  m = re.search("T(\d{3})", size_text)
  height = ""
  if m:
    height = m.group(1)
  
  m = re.search("B(\d+)", size_text)
  bust = m.group(1)

  m = re.search("W(\d+)", size_text)
  waist = m.group(1)

  m = re.search("H(\d+)", size_text)
  hips = m.group(1)

  breast_size = ""
  m = re.search(r"(\w)カップ", size_text)
  if m:
    breast_size = m.group(1)

  blood_type=""
  blood_type_text = b.find('span', text="血液型").find_parent("td").find("p").text
  m = re.search("(\w)型", blood_type_text)
  if m != None:
    blood_type = m.group(1)

  birth_place = b.find('span', text="出身地").find_parent("td").find("p").text.strip()
  
  av_career_text = b.find('span', text="AV出演期間").find_parent("td").find("p").text
  m = re.search(r"(.+)年.*(.*)年*", av_career_text)
  debut_year = ""
  retire_year = ""
  if m:
    debut_year = m.group(1)
    retire_year = m.group(2)

  interest = b.find('span', text="趣味・特技").find_parent("td").find("p").text.strip()

  debut_product = b.find('span', text="デビュー作品").find_parent("td").find("p").text.strip()

  template = Template("""{{AV_Basic
|imageFile1=$file_name.jpg
|imageFile2=${file_name}2.jpg
|imageFile3=${file_name}3.jpg
|imageFile4=${file_name}4.jpg
|enName=$name
|birthYear=$birth_year
|birthMonth=$birth_month
|birthDay=$birth_day
|birthPlace=$birth_place
|bloodType=$blood_type
|height=$height
|weight=$weight
|bust=$bust
|cup=$cup
|bustUnder=
|waist=$waist
|hips=$hips
|interest=$interest
|skill=$interest
|twitter=
|instagram=
|blog=
|website=
|wiki=$wiki
|debutYear=$debut_year
|retireYear=$retire_year
|alias=
|寫真女優=
|單體女優=
|企劃女優=
|企劃單體女優=
|無碼女優=
|脫衣舞孃=
|名人堂=
|ロリ系=
|巨乳系=
|已引退=
|低身長=
|長身=
|待完善=
}}

== 經歷 ==
出道作：$debut_product

== 作品 ==
=== Web ===
=== 寫真 ===

== Gallery ==
    """)
  print(template.substitute(birth_year=birth_year, 
                            birth_month=birth_month,
                            birth_day=birth_day,
                            birth_place=birth_place,
                            blood_type=blood_type,
                            height=height,
                            weight="",
                            cup=breast_size,
                            bust=bust,
                            waist=waist,
                            hips=hips,
                            wiki=url,
                            debut_year=debut_year,
                            retire_year=retire_year,
                            interest=interest,
                            name=name,
                            debut_product=debut_product,
                            file_name=name.replace(" ", "_")
                            ))

In [0]:
def parse_wiki(url, name):
  r = requests.get(url)
  if r.status_code == requests.codes.ok:
    b = BeautifulSoup(r.text)
    infobox = b.find_all("table", {"class": "infobox"})[0]
    birthday_obj = infobox.find("a", {"title": "誕生日"})
    birth_year = ""
    birth_month = ""
    birth_day = ""
    if birthday_obj:
      p = birthday_obj.find_parent("tr").find("td")
      birth_text = p.text
      m = re.search(r"(\d+)年(\d+)月(\d+)日", birth_text)
      birth_year = m.group(1)
      birth_month = m.group(2)
      birth_day = m.group(3)

    birth_place_obj = infobox.find("th", text="出身地")
    birth_place = ""
    if birth_place_obj:
      birth_place = birth_place_obj.find_parent("tr").find("td").text.strip()

    blood_type_obj = infobox.find("a", text="血液型")
    blood_type = ""
    if blood_type_obj:
      blood_type = blood_type_obj.find_parent("tr").find("td").text.strip()
      blood_type = blood_type.replace("型", "")
    
    hw_text = infobox.find("a", text="身長").find_parent("tr").find("td").text
    m = re.search(r"(.+)cm.+/(.+)kg", hw_text)
    
    height = m.group(1).strip()
    weight = m.group(2).strip()
    if not weight.isdigit():
      weight = ""

    three_size_text = infobox.find("a", {"title": "スリーサイズ"}).find_parent("tr").find("td").text
    m = re.search(r"(.+)-(.+)-(.+)cm", three_size_text)
    bust = m.group(1).strip()
    waist = m.group(2).strip()
    hips = m.group(3).strip()

    breast_size = ""
    breast_under = ""
    breast_size_obj = infobox.find("a", {"title": "ブラジャー"})
    if breast_size_obj != None: 
      breast_text = breast_size_obj.find_parent("tr").find("td").text.strip()
      breast_size = breast_text
      if "カップ" in breast_text:
        breast_size = breast_text.replace("カップ", "")
      
      if "-" in breast_text:
        breast_text2 = breast_text.split("-")
        breast_size = breast_text2[0]
        breast_under = breast_text2[1]

    av_career_text = ""
    debut_year = ""
    retire_year = ""
    av_career_obj = infobox.find("th", text="AV出演")
    if av_career_obj:
      av_career_text = av_career_obj.find_parent("tr").find("td").text
    
      m = re.search(r"(.+)年.*-{0,1}(.*)年*", av_career_text)
      if m:
        debut_year = m.group(1)
        retire_year = m.group(2)
        retire_year = retire_year.strip().replace("年", "")
      m = re.search(r"(.+)年.+月", av_career_text)
      if m:
        debut_year = m.group(1)
        retire_year = ""
    
    template = Template("""{{AV_Basic
|imageFile1=$file_name.jpg
|imageFile2=${file_name}2.jpg
|imageFile3=${file_name}3.jpg
|imageFile4=${file_name}4.jpg
|enName=$name
|birthYear=$birth_year
|birthMonth=$birth_month
|birthDay=$birth_day
|birthPlace=$birth_place
|bloodType=$blood_type
|height=$height
|weight=$weight
|bust=$bust
|cup=$cup
|bustUnder=$breast_under
|waist=$waist
|hips=$hips
|interest=
|skill=
|twitter=
|instagram=
|blog=
|website=
|wiki=$wiki
|debutYear=$debut_year
|retireYear=$retire_year
|alias=
|寫真女優=
|單體女優=
|企劃女優=
|企劃單體女優=
|無碼女優=
|脫衣舞孃=
|名人堂=
|ロリ系=
|巨乳系=
|已引退=
|低身長=
|長身=
|待完善=
}}

== 經歷 ==

== 作品 ==
=== Web ===
=== 寫真 ===

== Gallery ==
    """)
    print(template.substitute(birth_year=birth_year, 
                              birth_month=birth_month,
                              birth_day=birth_day,
                              birth_place=birth_place,
                              blood_type=blood_type,
                              height=height,
                              weight=weight,
                              cup=breast_size,
                              bust=bust,
                              waist=waist,
                              hips=hips,
                              breast_under=breast_under,
                              wiki=url,
                              debut_year=debut_year,
                              retire_year=retire_year,
                              name=name,
                              file_name=name.replace(" ", "_")
                              ))

In [0]:
def get_website(url, title):
  r = requests.get(url)
  #print(r.text)
  b = BeautifulSoup(r.text, 'html.parser')
  #print(b)
  adult_site_obj = b.find_all("h4")
  for obj in adult_site_obj:
    if re.search(title, obj.text):
      content_obj = obj.find_parent().find_next('div')
      for a in content_obj.find_all("a", {"class":"outlink"}, text=re.compile("^(?!\(レーベル|\(シリーズ|こちら|Blu-ray Disc版|\\.)")):
        print("* "+a.text)

In [0]:
parse_wiki("https://ja.wikipedia.org/wiki/菊川麻里", "Mari Kiku")
#parse_minanoav("http://www.minnano-av.com/actress71605.html", "Mio Honjo")

In [0]:
get_website("http://sougouwiki.com/d/%c3%e6%cc%ee%b6%b3%bb%d2", "アダルトサイト")

In [0]:
def parse_seesaawiki(url):
  r = requests.get(url);
  b = BeautifulSoup(r.text, 'html.parser')
  #pprint.PrettyPrinter(4).pprint(b)
  #link_objs = b.find_all("a", {"href": re.compile(r"https://seesaawiki.jp/av_neme/d/.")})
  div_objs = b.find_all("div", {"class": "wiki-section-3"});
  for div_obj in div_objs:
    link_obj = div_obj.find("a", {"href": re.compile(r"https://seesaawiki.jp/av_neme/d/.")})
    if link_obj:
      m = re.search(r"(.+)\((.+)\)", link_obj.text)
      if m:
        if m.group(2) != "仮":
          names = [m.group(1),m.group(2)]
        else:
          names = [m.group(1)]
        print(names)
      else:
        pass

  #pprint.PrettyPrinter(4).pprint(link_objs)
  # for link_obj in link_objs:
  #   print(link_obj.text)
  #   if re.search(r"\(.*\)",link_obj.text):
  #     #print(link_obj.text)
  #     pass

parse_seesaawiki("https://seesaawiki.jp/av_neme/d/S%2dCute")